# install requiremenets

In [ ]:
%pip install transformers
%pip install torch
%pip install pickle5

# Load pretrained Bert model and tokenizer

In [ ]:
import torch
from transformers import BertTokenizer, BertModel

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt
%matplotlib inline

# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )
model.eval()
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Open files and extract embedding from bert

In [ ]:
import pickle5 as pickle

def open_pklfile(filepath, size):
	with open(filepath, "rb") as f:
		if (size == 0):
			return pickle.load(f)
		return (pickle.load(f))[0:size]

def extract_bert_embeddings(word_list):
	#init for stacking embeddings
	embeddings = torch.empty(0)
	for word in word_list:
		
		if (word in '___'):
			continue
		# Map the token strings to their vocabulary indeces.
		tokenized_text = tokenizer.tokenize(word)
		
		# handling such as "wedding_dress"
		tokenized_text = [token for token in tokenized_text if token != '_']

		# Split the sentence into tokens.
		indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

		# Convert inputs to PyTorch tensors
		tokens_tensor = torch.tensor([indexed_tokens])
		
		# Put the model in "evaluation" mode,meaning feed-forward operation.
		model.eval()

		#Run the text through BERT, get the output and collect all of the hidden states produced from all 12 layers.
		with torch.no_grad():
			outputs = model(tokens_tensor)
			# can use last hidden state as word embeddings
			last_hidden_state = outputs[0]
			last_hidden_state = torch.sum(last_hidden_state, 1, True)
			embeddings = torch.cat([embeddings, last_hidden_state], dim = 1)

	print(embeddings.shape)
	return torch.squeeze(embeddings)

# TODO: Bias computing with BERT

# Experiment 1 : Clustering

In [ ]:
# Auxiliary finctions

import matplotlib as mpl
import mpld3
from cycler import cycler
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import sys

from sklearn.cluster import KMeans
#from sklearn.datasets import make_blobs

%matplotlib inline
mpld3.enable_notebook()
mpl.rc("savefig", dpi=200)
mpl.rcParams['figure.figsize'] = (8,8)
mpl.rcParams['axes.prop_cycle'] = cycler(color='rc')


def visualize(vectors, words, labels, ax, title, random_state, num_clusters = 2):
    
    # perform TSNE
    X_embedded = TSNE(n_components=2, random_state=random_state).fit_transform(vectors)
    if num_clusters == 2:
        for x,l in zip(X_embedded, labels):
            if l:
                ax.scatter(x[0], x[1], marker = '.', c = 'c')
            else:
                ax.scatter(x[0], x[1], marker = 'x', c = 'darkviolet')
    else:
        ax.scatter(X_embedded[:,0], X_embedded[:,1], c = labels)                
    
    ax.text(.01, .9, title ,transform=ax.transAxes, fontsize=18)

    
def extract_vectors(words, space1 = 'limit_bef', space2 = 'limit_aft'):
    
    size = len(words)/2
    
    X_bef = [wv[space1][w2i[space1][x],:] for x in words]
    X_aft = [wv[space2][w2i[space2][x],:] for x in words]

    return X_bef, X_aft


def cluster_and_visualize(words, X_bef, X_aft, random_state, y_true, num=2):

    fig, axs = plt.subplots(1, 2, figsize=(15, 3))
    
    y_pred_bef = KMeans(n_clusters=num, random_state=random_state).fit_predict(X_bef)
    visualize(X_bef, words, y_pred_bef, axs[0], 'Original', random_state)
    correct = [1 if item1 == item2 else 0 for (item1,item2) in zip(y_true, y_pred_bef) ]
    print ('precision bef', sum(correct)/float(len(correct)))
    
    y_pred_aft = KMeans(n_clusters=num, random_state=random_state).fit_predict(X_aft)
    visualize(X_aft, words, y_pred_aft, axs[1], 'Debiased', random_state)
    correct = [1 if item1 == item2 else 0 for (item1,item2) in zip(y_true, y_pred_aft) ]
    print ('precision aft', sum(correct)/float(len(correct)))
    fig.show()
    fig.savefig("2016_clustering figure", bbox_inches='tight')

In [ ]:
# Cluster most biased words before and after debiasing
import operator

size = 500
top500_male_words = open_pklfile("../data/lists/top2500male_2016.pkl", 500)
top500_male_embeddings = extract_bert_embeddings(top500_male_words)

top500_female_words = open_pklfile("../data/lists/top2500female_2016.pkl", 500)
top500_fe_embeddings = extract_bert_embeddings(top500_female_words)

random_state = 1

# sorting with bert bias
# sorted_g = sorted(gender_bias_bef.items(), key=operator.itemgetter(1))
# female = [item[0] for item in sorted_g[:size]]
# male = [item[0] for item in sorted_g[-size:]]

# X_bef, X_aft = extract_vectors(male + female)
y_true = [1]*size + [0]*size
cluster_and_visualize(top500_male_words + top500_female_words,
 top500_male_embeddings + top500_fe_embeddings,
  top500_male_embeddings + top500_fe_embeddings,
   random_state, y_true)

# TODO: Experiment 2 : Professions

# Experiment 3 : Classification with SVM

In [ ]:
size = 2500
top2500_male_words = open_pklfile("../data/lists/top2500male_2016.pkl", 0)
top2500_male_embeddings = extract_bert_embeddings(top2500_male_words)

top2500_female_words = open_pklfile("../data/lists/top2500female_2016.pkl", 0)
top2500_fe_embeddings = extract_bert_embeddings(top2500_female_words)

In [ ]:
# take 5000 most biased words, split each polarity randomly to train (1/5) and test (4/5), and predict

from sklearn import svm
from random import shuffle
import random
random.seed(10)

def train_and_predict(male_embeddings, female_embeddings):
    size_train = 500
    size_test = 1999

    # X_train = [wv[space_train][w2i[space_train][w],:] for w in males[:size_train]+females[:size_train]]
    # Y_train = [1]*size_train + [0]*size_train
    # X_test = [wv[space_test][w2i[space_test][w],:] for w in males[size_train:]+females[size_train:]]
    # Y_test = [1]*size_test + [0]*size_test

    total = size_train+size_test
    X_train = torch.cat([male_embeddings[:size_train], female_embeddings[:size_train]], dim = 0)
    Y_train = [1]*size_train + [0]*size_train

    X_test = torch.cat([male_embeddings[size_train:total], female_embeddings[size_train:total]], dim = 0)
    Y_test = [1]*size_test + [0]*size_test

    print("train set shape is ", X_train.shape)
    print("test set shape is ", X_test.shape)
    clf = svm.SVC()

    clf.fit(X_train, Y_train)

    preds = clf.predict(X_test)

    accuracy = [1 if y==z else 0 for y,z in zip(preds, Y_test)]
    print ('accuracy:', float(sum(accuracy))/len(accuracy))

# size = size_train + size_test
# sorted_g = sorted(gender_bias_bef.items(), key=operator.itemgetter(1))
# females = [item[0] for item in sorted_g[:size]]
# males = [item[0] for item in sorted_g[-size:]]
# for f in females:
#     assert(gender_bias_bef[f] < 0)
# for m in males:
#     assert(gender_bias_bef[m] > 0)


shuffle(top2500_male_embeddings)
shuffle(top2500_fe_embeddings)

# classification before debiasing

train_and_predict(top2500_male_embeddings, top2500_fe_embeddings)

# classification after debiasing

# train_and_predict('aft', 'aft')
